# 数学优化建模简介

## 目标和预备知识

你是否想学习数学优化建模的基础知识?如果是,那么这是一个很好的起点。在本教程中,我们将带你了解构建数学优化模型和求解数学优化问题的过程。我们将首先概述一个简单数学优化问题的关键组成部分,然后展示如何使用 Gurobi Python API 创建该问题的数学优化模型(更准确地说,是混合整数规划或 MIP 模型),最后演示如何使用 Gurobi 求解器自动生成最优解。

本建模教程属于入门级别,我们假设你了解 Python 并具有使用定量方法的学科背景。

建议你参考 Gurobi Python API 的[文档](https://www.gurobi.com/resources/?category-filter=documentation)。
本笔记本在我们的混合整数线性规划系列教程视频中有详细解释。你可以点击[此处](https://www.gurobi.com/resource/tutorial-mixed-integer-linear-programming/)观看这些视频。

**下载代码库** <br />
你可以点击[此处](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含本示例和其他示例的代码库。

## 问题描述

考虑一家咨询公司有三个空缺职位:测试员、Java开发人员和架构师。这些职位的三个顶级候选人(资源)是:Carlos、Joe和Monika。咨询公司对每个候选人进行了能力测试,以评估他们执行每项工作的能力。这些测试的结果被称为*匹配分数*。假设只能将一名候选人分配到一个职位,且每个候选人最多只能被分配一个职位。

问题是要确定资源和职位的分配方式,使得每个职位都得到满足,每个资源最多分配一个职位,并且分配的总匹配分数最大化。


## 数学优化

数学优化(也称为数学规划)是一种声明式方法,建模者通过该方法制定能捕获复杂决策问题关键特征的优化问题。Gurobi求解器使用最先进的数学和计算机科学技术来求解数学优化问题。

数学优化模型有五个组成部分:

* 集合
* 参数
* 决策变量
* 约束条件
* 目标函数

In [1]:
%pip install gurobipy

下面的Python代码导入Gurobi可调用库,并将`GRB`类导入到主命名空间中。

In [1]:
import gurobipy as gp
from gurobipy import GRB

## 资源分配问题
### 数据
列表 $R$ 包含三个资源的名称:Carlos、Joe和Monika。

列表 $J$ 包含职位名称:测试员、Java开发人员和架构师。

$r \in R$:资源的索引和集合。资源 $r$ 属于资源集合 $R$。

$j \in J$:职位的索引和集合。职位 $j$ 属于职位集合 $J$。

In [2]:
# 资源和职位集合
R = ['Carlos', 'Joe', 'Monika']
J = ['Tester', 'JavaDeveloper', 'Architect']

下面的匹配分数表列出了每个资源执行每项工作的能力:

![分数](util/matching_score_data.PNG)

对于每个资源 $r$ 和职位 $j$,都有一个对应的匹配分数 $s$。匹配分数 $s$ 只能取0到100之间的值。即对于所有资源 $r \in R$ 和职位 $j \in J$,有 $s_{r,j} \in [0, 100]$。 

我们使用Gurobi Python的 `multidict` 函数通过一个语句初始化一个或多个字典。该函数以字典作为参数。键表示资源和职位的可能组合。

In [3]:
# 匹配分数数据
combinations, scores = gp.multidict({
    ('Carlos', 'Tester'): 53,
    ('Carlos', 'JavaDeveloper'): 27,
    ('Carlos', 'Architect'): 13,
    ('Joe', 'Tester'): 80,
    ('Joe', 'JavaDeveloper'): 47,
    ('Joe', 'Architect'): 67,
    ('Monika', 'Tester'): 53,
    ('Monika', 'JavaDeveloper'): 73,
    ('Monika', 'Architect'): 47
})

以下构造函数创建一个空的`Model`对象"m"。我们通过传递字符串"RAP"作为参数来指定模型名称。`Model`对象"m"包含一个优化问题。它由一组变量、一组约束和目标函数组成。

In [4]:
# 声明和初始化模型
m = gp.Model('RAP')

Set parameter LicenseID to value 2601452


## 决策变量

为了解决这个分配问题,我们需要确定哪个资源分配给哪个职位。我们针对资源和职位的每种可能分配都引入一个决策变量。因此,我们有9个决策变量。

为了简化模型公式的数学表示,我们为资源和职位定义以下索引:

![变量](util/decision_variables.PNG)

例如,$x_{2,1}$是与将资源Joe分配给测试员职位相关的决策变量。因此,如果资源 $r \in R$ 被分配给职位 $j \in J$,则决策变量$x_{r,j}$等于1,否则等于0。

`Model.addVars()`方法为`Model`对象创建决策变量。该方法返回一个包含新创建变量的Gurobi `tupledict`对象。我们将`combinations`对象作为第一个参数来指定变量索引。`name`关键字用于为新创建的决策变量指定名称。默认情况下,变量被假定为非负的。

In [5]:
# 为RAP模型创建决策变量
x = m.addVars(combinations, name="assign")

## 职位约束

现在我们讨论与职位相关的约束。这些约束需要确保每个职位恰好由一个资源填补。

测试员职位的约束要求资源1(Carlos)、资源2(Joe)或资源3(Monika)被分配到这个职位。这对应于以下约束。

约束(测试员=1)

$$
x_{1,1} + x_{2,1} + x_{3,1} = 1
$$

同样,Java开发人员和架构师职位的约束可以定义如下。

约束(Java开发人员=2)

$$
x_{1,2} + x_{2,2} + x_{3,2} = 1
$$

约束(架构师=3)

$$
x_{1,3} + x_{2,3} + x_{3,3} = 1
$$

职位约束由下表的列定义。

![职位](util/jobs_constraints.PNG)

一般来说,测试员职位的约束可以定义如下。

$$
x_{1,1} + x_{2,1} + x_{3,1} = \sum_{r=1}^{3 } x_{r,1} =  \sum_{r \in R} x_{r,1} = 1
$$

所有的职位约束都可以用类似简洁的方式定义。对于每个职位 $j \in J$,对所有资源的决策变量求和。我们可以将相应的职位约束写成如下形式。

$$
\sum_{r \in R} x_{r,j} = 1
$$

`Model.addConstrs()`方法定义了`Model`对象“m”的职位约束。该方法返回一个包含职位约束的Gurobi `tupledict`对象。
该方法的第一个参数"x.sum(‘*’, j)"是求和方法,定义了职位约束的左侧:
对于职位集合 $J$ 中的每个职位 $j$,对所有资源的决策变量求和。$==$ 定义了一个等式约束,数字"1"是约束的右侧。
这些约束表示每个职位恰好由一个资源填补。
第二个参数是这种类型约束的名称。


In [6]:
# 创建职位约束
jobs = m.addConstrs((x.sum('*',j) == 1 for j in J), name='job')

## 资源约束

资源的约束需要确保每个资源最多被分配一个职位。也就是说,可能并不是所有的资源都被分配。

例如,我们希望一个约束要求Carlos最多被分配一个职位:要么是职位1(测试员),要么是职位2(Java开发人员),要么是职位3(架构师)。我们可以将这个约束写成如下形式。

约束(Carlos=1)

$$
x_{1, 1} + x_{1, 2} + x_{1, 3}  \leq 1.
$$

这个约束小于或等于1,以允许Carlos不被分配任何职位。同样,Joe和Monika的资源约束可以定义如下:

约束(Joe=2) 

$$
x_{2, 1} + x_{2, 2} + x_{2, 3}  \leq 1.
$$

约束(Monika=3)

$$
x_{3, 1} + x_{3, 2} + x_{3, 3}  \leq 1.
$$

注意,资源约束由下表的行定义。

![资源](util/resource_constraints.PNG)

Carlos的资源约束可以定义如下。

$$
x_{1, 1} + x_{1, 2} + x_{1, 3} = \sum_{j=1}^{3 } x_{1,j} = \sum_{j \in J} x_{1,j} \leq 1.
$$

同样,每个约束都可以用简洁的方式写成。对于每个资源 $r \in R$,对所有职位的决策变量求和。我们可以将相应的资源约束写成如下形式。

$$
\sum_{j \in J} x_{r,j} \leq  1.
$$

`Model.addConstrs()`方法定义了`Model`对象“m”的资源约束。
该方法的第一个参数"x.sum(r, ‘*’)"是求和方法,定义了资源约束的左侧:对于资源集合 $R$ 中的每个资源 $r$,对所有职位的决策变量求和。
$<=$ 定义了一个小于或等于的约束,数字"1"是约束的右侧。
这些约束表示每个资源最多被分配一个职位。
第二个参数是这种类型约束的名称。


In [7]:
# 创建资源约束
resources = m.addConstrs((x.sum(r,'*') <= 1 for r in R), name='resource')

## 目标函数

目标函数是最大化满足职位和资源约束的分配的总匹配分数。

对于测试员职位,如果资源Carlos被分配,匹配分数是$53x_{1,1}$,如果资源Joe被分配,匹配分数是$80x_{2,1}$,如果资源Monika被分配,匹配分数是$53x_{3,1}$。
因此,测试员职位的匹配分数如下,其中只有一个项在这个求和中为非零。

$$
53x_{1,1} + 80x_{2,1} + 53x_{3,1}. 
$$

同样,Java开发人员和架构师职位的匹配分数定义如下。Java开发人员职位的匹配分数是:

$$
27x_{1, 2} + 47x_{2, 2} + 73x_{3, 2}.
$$

架构师职位的匹配分数是:

$$
13x_{1, 3} + 67x_{2, 3} + 47x_{3, 3}.
$$

总匹配分数是下表中每个单元格的求和。

![目标函数](util/objective_function.PNG)

目标是最大化分配的总匹配分数。因此,目标函数定义如下。

\begin{equation}
\text{Maximize} \quad (53x_{1,1} + 80x_{2,1} + 53x_{3,1}) \; +
\end{equation}

\begin{equation}
\quad (27x_{1, 2} + 47x_{2, 2} + 73x_{3, 2}) \; +
\end{equation}

\begin{equation}
\quad (13x_{1, 3} + 67x_{2, 3} + 47x_{3, 3}).
\end{equation}

目标函数中的每个括号中的项可以表示如下。

\begin{equation}
(53x_{1,1} + 80x_{2,1} + 53x_{3,1}) = \sum_{r \in R} s_{r,1}x_{r,1}.
\end{equation}

\begin{equation}
(27x_{1, 2} + 47x_{2, 2} + 73x_{3, 2}) = \sum_{r \in R} s_{r,2}x_{r,2}.
\end{equation}

\begin{equation}
(13x_{1, 3} + 67x_{2, 3} + 47x_{3, 3}) = \sum_{r \in R} s_{r,3}x_{r,3}.
\end{equation}

因此,目标函数可以简洁地写成:

\begin{equation}
\text{Maximize} \quad \sum_{j \in J} \sum_{r \in R} s_{r,j}x_{r,j}.
\end{equation}

`Model.setObjective()`方法定义了`Model`对象“m”的目标函数。目标表达式在该方法的第一个参数中指定。
注意,匹配分数参数“score”和分配决策变量“x”都是在“combinations”键上定义的。因此,我们使用方法“x.prod(score)”来获得“score”矩阵和“x”变量矩阵的元素乘积的求和。
第二个参数``GRB.MAXIMIZE``是优化的"方向"。在这种情况下,我们希望*最大化*所有分配的总匹配分数。

In [8]:
# 目标：最大化所有分配的总匹配分数
m.setObjective(x.prod(scores), GRB.MAXIMIZE)

我们使用Gurobi/Python API的“write()”方法将模型公式写入名为"RAP.lp"的文件。

In [ ]:
# 保存模型以供检查
m.write('RAP.lp')

![RAP](util/RAP_lp.PNG)

我们使用Gurobi/Python API的“optimize()”方法来求解我们为模型对象“m”定义的问题。

In [9]:
# 运行优化引擎
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xb343b6eb
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6000000e+32   1.800000e+31   4.600000e+02      0s
       5    1.9300000e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.930000000e+02


`Model.getVars()`方法检索模型对象“m”中的所有变量列表。`.x`变量属性用于查询解值,`.varName`属性用于查询决策变量的名称。

In [10]:
# 显示决策变量的最优值
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# 显示总匹配分数
print('Total matching score: ', m.objVal)

assign[Carlos,Tester] 1.0
assign[Joe,Architect] 1.0
assign[Monika,JavaDeveloper] 1.0
Total matching score:  193.0


最优分配是将:

* Carlos分配给测试员职位,匹配分数为53
* Joe分配给架构师职位,匹配分数为67
* Monika分配给Java开发人员职位,匹配分数为73。

最大总匹配分数为193。

## 带预算约束的资源分配问题

现在,假设分配资源 $r \in R$ 到职位 $j \in J$ 需要一个固定成本 $C_{r,j}$。还假设有一个有限的预算 $B$ 可以用于职位分配。

将Carlos、Joe或Monika分配到任何职位的成本分别为$\$1,000$ , $\$2,000$ , 和 $\$3,000$。可用预算为$\$5,000$。

### 数据

列表 $R$ 包含三个资源的名称:Carlos、Joe和Monika。
列表 $J$ 包含职位名称:测试员、Java开发人员和架构师。

Gurobi Python的`multidict`函数初始化两个字典:
* "scores"定义了每个资源和职位组合的匹配分数。
* "costs"定义了将资源分配到职位的固定成本。



In [11]:
# 资源和职位集合
R = ['Carlos', 'Joe', 'Monika']
J = ['Tester', 'JavaDeveloper', 'Architect']

# 匹配分数数据
# 成本以千美元为单位
combinations, scores, costs = gp.multidict({
    ('Carlos', 'Tester'): [53, 1],
    ('Carlos', 'JavaDeveloper'): [27, 1],
    ('Carlos', 'Architect'): [13,1],
    ('Joe', 'Tester'): [80, 2],
    ('Joe', 'JavaDeveloper'): [47, 2],
    ('Joe', 'Architect'): [67, 2],
    ('Monika', 'Tester'): [53, 3] ,
    ('Monika', 'JavaDeveloper'): [73, 3],
    ('Monika', 'Architect'): [47, 3]
})

# 可用预算(千美元)
budget = 5

以下构造函数创建一个空的`Model`对象“m”。`Model`对象“m”包含一个优化问题。它由一组变量、一组约束和目标函数组成。

In [12]:
# 声明和初始化模型
m = gp.Model('RAP2')

### 决策变量

决策变量 $x_{r,j}$ 等于1,如果 $r \in R$ 被分配到职位 $j \in J$,否则等于0。

`Model.addVars()`方法定义了模型对象“m”的决策变量。

由于存在预算约束,可能并不是所有的职位都能填补。为了解决这个问题,我们定义一个新决策变量,表示职位是否被填补。

令 $g_{j}$ 等于1,如果职位 $j \in J$ 没有被填补,否则等于0。这个变量是一个间隙变量,表示职位无法填补。

***备注:*** 对于之前的RAP公式,我们将分配变量定义为非负和连续的,这是`Model.addVars()`方法的默认值。
然而,在RAP的这个扩展中,由于我们在模型中添加了预算约束,我们需要显式地将这些变量定义为二进制。`Model.addVars()`方法的`vtype=GRB.BINARY`参数将分配变量定义为二进制。

In [13]:
# 为RAP模型创建决策变量
x = m.addVars(combinations, vtype=GRB.BINARY, name="assign")

# 为RAP模型创建间隙变量
g = m.addVars(J, name="gap")

### 职位约束

由于我们有一个有限的预算来分配资源到职位,可能并不是所有的职位都能填补。对于职位约束,有两种可能性:要么分配一个资源来填补职位,要么这个职位无法填补,我们需要声明一个间隙。后一种可能性由决策变量 $g_j$ 捕获。因此,职位约束写成如下形式。

对于每个职位 $j \in J$,必须恰好分配一个资源到职位,或者相应的 $g_j$ 变量必须设置为1:

$$
\sum_{r \: \in \: R} x_{r,\; j} + g_{j} = 1.
$$


In [14]:
# 创建职位约束
jobs = m.addConstrs((x.sum('*',j) + g[j]  == 1 for j in J), name='job')

### 资源约束

资源的约束需要确保每个资源最多被分配一个职位。也就是说,可能并不是所有的资源都被分配。因此,资源约束写成如下形式。

对于每个资源 $r \in R$,最多可以分配一个职位给资源:

$$
\sum_{j \: \in \: J} x_{r,\; j} \leq 1.
$$

In [15]:
# 创建资源约束
resources = m.addConstrs((x.sum(r,'*') <= 1 for r in R), name='resource')

### 预算约束

这个约束确保分配资源以满足职位需求的成本不超过可用预算。分配和预算的成本以千美元为单位。

填补测试员职位的成本是$1x_{1,1}$,如果资源Carlos被分配,或者$2x_{2,1}$,如果资源Joe被分配,或者$3x_{3,1}$,如果资源Monika被分配。
因此,填补测试员职位的成本如下,其中最多有一个项在这个求和中为非零。

$$
1x_{1,1} + 2x_{2,1} + 3x_{3,1}. 
$$

同样,填补Java开发人员和架构师职位的成本定义如下。填补Java开发人员职位的成本是:

$$
1x_{1, 2} + 2x_{2, 2} + 3x_{3, 2}.
$$

填补架构师职位的成本是:

$$
1x_{1, 3} + 2x_{2, 3} + 3x_{3, 3}.
$$

因此,填补职位的总成本应小于或等于可用预算。

\begin{equation}
(1x_{1,1} + 2x_{2,1} + 3x_{3,1}) \; +
\end{equation}

\begin{equation}
(1x_{1, 2} + 2x_{2, 2} + 3x_{3, 2}) \; +
\end{equation}

\begin{equation}
(1x_{1, 3} + 2x_{2, 3} + 3x_{3, 3}) \leq 5
\end{equation}

预算约束中的每个括号中的项可以表示如下。

\begin{equation}
(1x_{1,1} + 2x_{2,1} + 3x_{3,1}) = \sum_{r \in R} C_{r,1}x_{r,1}.
\end{equation}

\begin{equation}
(1x_{1, 2} + 2x_{2, 2} + 3x_{3, 2}) = \sum_{r \in R} C_{r,2}x_{r,2}.
\end{equation}

\begin{equation}
(1x_{1, 3} + 2x_{2, 3} + 3x_{3, 3}) = \sum_{r \in R} C_{r,3}x_{r,3}.
\end{equation}

因此,预算约束可以简洁地写成:

\begin{equation}
\sum_{j \in J} \sum_{r \in R} C_{r,j}x_{r,j} \leq B.
\end{equation}

`Model.addConstr()`方法定义了`Model`对象“m”的预算约束。
该方法的第一个参数"x.prod(costs)"是prod方法,定义了预算约束的左侧。$<=$ 定义了一个小于或等于的约束,可用预算金额是约束的右侧。
这个约束表示分配资源以满足职位需求的总成本不能超过可用预算。
第二个参数是这个约束的名称。

In [16]:
budget = m.addConstr((x.prod(costs) <= budget), name='budget')

## 目标函数

目标函数类似于RAP。目标的第一个项是分配的总匹配分数。在RAP的这个扩展中,可能并不是所有的职位都能填补;然而,我们希望对此可能性进行重罚。为此,我们在目标函数中有第二个项,对所有职位的间隙变量求和,并乘以一个大罚值$M$。

注意,匹配分数的最大值是100,我们给$M$的值是101。$M$值的依据是,有间隙会严重降低总匹配分数值。

因此,目标函数是最大化分配的总匹配分数减去间隙变量值为1的罚值。

$$
\max \; \sum_{j \; \in \; J} \sum_{r \; \in \; R} s_{r,j}x_{r,j} -M \sum_{j \in J} g_{j}
$$

In [17]:
# 没有填补职位空缺的处罚
M = 101

In [18]:
# 目标：使作业的总匹配分数最大化
# 空缺职位会受到严重惩罚
m.setObjective(x.prod(scores) - M*g.sum(), GRB.MAXIMIZE)

In [19]:
# 运行优化引擎
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 7 rows, 12 columns and 30 nonzeros
Model fingerprint: 0xa1231a12
Variable types: 3 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 30 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)
Found heuristic solution: objective 52.0000000

Root relaxation: objective 1.350000e+02, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  135.00000    0    2   52.00000  135.00000   160

目标函数的定义包括不填补职位的罚值。然而,我们对未填补所有职位时的最优总匹配分数值感兴趣。为此,我们需要使用匹配分数值$s_{r,j}$和分配决策变量$x_{r,j}$来计算总匹配分数值。

In [20]:
# 从分配变量计算总匹配分数
total_matching_score = 0
for r, j in combinations:
    if x[r, j].x > 1e-6:
        print(x[r, j].varName, x[r, j].x) 
        total_matching_score += scores[r, j]*x[r, j].x

print('Total matching score: ', total_matching_score)  

assign[Joe,Tester] 1.0
assign[Monika,JavaDeveloper] 1.0
Total matching score:  153.0


### 分析

回顾一下,预算是$\$5,000$,分配三个资源的总成本是$\$6,000$。这意味着没有足够的预算来分配我们拥有的三个资源。因此,Gurobi优化器必须选择两个资源来填补职位需求,留下一个职位未填补,并最大化总匹配分数。注意,两个最高匹配分数是80%(Joe为测试员职位)和73%(Monika为Java开发人员职位)。还要注意,最低分数是13%(Carlos为架构师职位)。将Joe分配给测试员职位,Monika分配给Java开发人员职位,没有人分配给架构师职位的成本是$\$5,000$,总匹配分数是153。这是Gurobi优化器找到的最优解。

In [21]:
m.dispose()
gp.disposeDefaultEnv()

Freeing default Gurobi environment


Copyright © 2020 Gurobi Optimization, LLC